In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os 
import sys
import time 
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for model in mpl, np, pd, sklearn, tf, keras:
    print(model.__name__, model.__version__)

In [ ]:
train_df = pd.read_csv("./input/train.csv")

In [ ]:
#Seaborn是基于matplotlib的图形可视化python包
import seaborn as sns

sns.set(style='white', context='notebook', palette='deep')

In [ ]:
#divide x, y from train_df

y_train = train_df['label']

#drop label column   axis = 1 == col; axis = 0 == row
x_train = train_df.drop(labels = ['label'], axis = 1)

#free space
del train_df

g = sns.countplot(y_train)

y_train.value_counts()

In [ ]:
#check the data
x_train.isnull().any().describe()

In [ ]:
test_df.isnull().any().describe()

In [ ]:
import pprint

pprint.pprint(x_train.shape)
pprint.pprint(test_df.shape)


In [ ]:
#Normalization

x_train = x_train / 255.0

pprint.pprint(x_train.shape)
pprint.pprint(test_df.shape)

In [ ]:
# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)

x_train = x_train.values.reshape(-1, 28, 28, 1)
test_df = test_df.values.reshape(-1, 28, 28, 1)

pprint.pprint(x_train.shape)
pprint.pprint(test_df.shape)

In [ ]:
# Encode labels to one hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
y_train = keras.utils.to_categorical(y_train, num_classes = 21)

In [ ]:
np.random.seed(2)
random_seed = 2

from sklearn.model_selection import train_test_split
# Split the train and the validation set for the fitting
# 10% for validation 90% for train

x_train, x_valid, y_train, y_valid = train_test_split(
    x_train, y_train, test_size = 0.1, random_state=random_seed)

pprint.pprint(x_train.shape)
pprint.pprint(x_valid.shape)

In [ ]:
#show example
g = plt.imshow(x_train[2][:,:,0])

In [ ]:
# Set the CNN model 
# my CNN architechture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out
#from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D

model = keras.Sequential()

model.add(keras.layers.Conv2D(filters = 32, kernel_size = 5, padding = 'same',
                 activation = 'relu', input_shape = (28, 28, 1)))
model.add(keras.layers.Conv2D(filters = 32, kernel_size = 5, padding = 'same',
                 activation = 'relu'))
model.add(keras.layers.MaxPool2D(pool_size = 2))
model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(filters = 64, kernel_size = 5, padding = 'same',
                 activation = 'relu'))
model.add(keras.layers.Conv2D(filters = 64, kernel_size = 5, padding = 'same',
                 activation = 'relu'))
model.add(keras.layers.MaxPool2D(pool_size = 2))
model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(255, activation = 'relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(21, activation = 'softmax'))

model.summary()

In [ ]:
# Define the optimizer
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [ ]:
# Compile the model
model.compile(optimizer = optimizer ,
              loss = "categorical_crossentropy",
              metrics=["accuracy"])

In [ ]:
epochs = 30
batch_size = 86

In [ ]:
# With data augmentation to prevent overfitting (accuracy 0.99286)
# from keras.preprocessing.image import ImageDataGenerator

datagen = keras.preprocessing.image.ImageDataGenerator(
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1)  # randomly shift images vertically (fraction of total height)

datagen.fit(x_train)

In [ ]:
train_generator = datagen.flow(x_train, y_train, batch_size=batch_size)

In [ ]:
logdir = './callbacks'
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir,
                                 "mnist_model.h5")

callbacks = [
keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', 
                                  patience=3, 
                                  verbose=1, 
                                  factor=0.5, 
                                  min_lr=0.00001)
]

In [ ]:
# Fit the model
history = model.fit_generator(train_generator,
                              epochs = epochs,
                              steps_per_epoch = x_train.shape[0] // batch_size,
                              validation_data = (x_valid, y_valid),
                              validation_steps = x_valid.shape[0] // batch_size,
                              callbacks = callbacks)

In [ ]:
def plot_learning_curves(history, label, epochs, min_value, max_value):
    data = {}
    data[label] = history.history[label]
    data['val_' + label] = history.history['val_' + label]
    pd.DataFrame(data).plot(figsize = (8, 5))
    plt.grid(True)
    plt.axis([0, epochs, min_value, max_value])
    plt.show()

plot_learning_curves(history, 'accuracy', epochs, 0.9, 1)
plot_learning_curves(history, 'loss', epochs, 0, 0.5)

In [ ]:
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize = False,
                          title = 'Confusion Matrix',
                          cmap = plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation = 'nearest', cmap = cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation = 45)
    plt.yticks(tick_marks, classes)
    
    if normalize:
        cm = cm.astype('float') / cm.sum(asix = 1)[:,np.newaxis]
        
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j], 
                 horizontalalignment = 'center',
                 color = 'white' if cm[i, j] > thresh else 'black')
        
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
# predict the values in validation dataset
y_pred = model.predict(x_valid)
# convert predictions classes to one hot vectors
# argmax is position of max data
y_pred_classes = np.argmax(y_pred, axis = 1)
# convert validation observations to one hot vectors
y_true = np.argmax(y_valid, axis = 1)
# comput the confusion matrix
confusion_matrix = sklearn.metrics.confusion_matrix(y_true, y_pred_classes)
# plot the confusion matrix
plot_confusion_matrix(confusion_matrix, classes = range(10))

In [ ]:
# Display some error results 

# Errors are difference between predicted labels and true labels
errors = (y_pred_classes - y_true != 0)

y_pred_classes_errors = y_pred_classes[errors]
y_pred_errors = y_pred[errors]
y_true_errors = y_true[errors]
x_valid_errors = x_valid[errors]

def display_errors(error_index, img_errors, pred_errors, obs_errors):
    """ This function shows 6 images with their predicted and real labels"""
    n = 0
    nrows = 2
    ncols = 3
    fig, ax = plt.subplots(nrows, ncols, sharex = True, sharey = True, figsize=(10, 7))
    for row in range(nrows):
        for col in range(ncols):
            error = error_index[n]
            ax[row][col].imshow((img_errors[error]).reshape((28, 28)))
            ax[row][col].set_title("Predicted label :{}\nTrue label :{}".format(
                pred_errors[error], obs_errors[error]))
            n += 1

# Probabilities of wrong predicted numbers
y_pred_errors_prob = np.max(y_pred_errors, axis = 1)

# a = [0, 1, 1, 2]
# b = np.diagonal(np.take([[0, 1, 8], [2, 3, 9], [4, 5, 10], [6, 7, 11]], a, axis = 1))
# pprint.pprint(b)
# Predicted probabilities of the true values in the error set
true_prob_errors = np.diagonal(np.take(y_pred_errors, y_true_errors, axis = 1))

# Different between the probabilities of predicetd label and true label
delta_pred_true_errors = y_pred_errors_prob - true_prob_errors

# Sorted list of the delta prob errors
sorted_delta_errors = np.argsort(delta_pred_true_errors)

# Top 6 errors
most_important_errors = sorted_delta_errors[-6:]

# Show the top 6 errors
display_errors(most_important_errors, x_valid_errors, y_pred_classes_errors, y_true_errors)

In [ ]:
model.save('my_model.h5')